In [1]:
import numpy as np
import random
import time
import imp
import json
import sklearn as sk
import random
from random import seed
from random import random
from random import randrange
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import json
import csv
import unicodecsv
import codecs
from collections import defaultdict
import sys
import csv
#imp.reload(sys)
#sys.stdout = codecs.getwriter('utf-8')(sys.stdout)
#sys.stderr = codecs.getwriter('utf-8')(sys.stderr)
import pickle
from collections import Counter
import itertools
import nltk
import re
from nltk.probability import *
import pandas as pd

## 1. Loading Data 

### a) yelp_academic_dataset_business_train.txt

In [315]:
business_train = []
with open('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_business_train.csv',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        business_train.append(tuple(line))

In [316]:
len(business_train)


2511

In [317]:
len(business_train[0]) == len(business_train[1])

True

In [329]:
business_train[1:len(business_train)][0]

('85',
 '',
 'Ja4stXdNYr39u5CZHMNtjw',
 "['Monday 11:0-23:0', 'Tuesday 11:0-0:0', 'Wednesday 11:0-23:0', 'Thursday 11:0-23:0', 'Friday 11:0-0:0', 'Saturday 11:0-0:0', 'Sunday 11:0-23:0']",
 '1',
 '7228 E 1st Ave',
 '[\'Alcohol: full_bar\', "Ambience: {\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB: False\', \'BYOBCorkage: no\', \'BikeParking: True\', \'BusinessAcceptsCreditCards: True\', "BusinessParking: {\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'Caters: True\', \'GoodForKids: True\', "GoodForMeal: {\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'breakfast\': False, \'brunch\': False}", \'HasTV: True\', \'NoiseLevel: average\', \'OutdoorSeating: True\', \'RestaurantsAttire: casual\', \'RestaurantsDelivery: False\', \'RestaurantsGoodForGroups: True\', \'Restaurants

### b) yelp_academic_dataset_checkin_train.txt

In [10]:
checkin_train = []
with open('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_checkin.csv',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        checkin_train.append(tuple(line))

In [11]:
len(checkin_train)

2878

In [12]:
checkin_train[0]

('', 'time', 'type', 'business_id')

In [13]:
checkin_train[1]

('402',
 "['Fri-0:5', 'Mon-0:3', 'Sun-0:2', 'Thu-0:4', 'Tue-0:1', 'Wed-0:4', 'Fri-1:3', 'Mon-1:1', 'Sat-1:1', 'Sun-1:4', 'Thu-1:6', 'Tue-1:2', 'Wed-1:4', 'Sun-2:1', 'Tue-2:3', 'Mon-3:1', 'Fri-4:1', 'Tue-4:1', 'Tue-7:1', 'Fri-9:31', 'Mon-9:53', 'Sat-9:27', 'Sun-9:29', 'Thu-9:37', 'Tue-9:37', 'Wed-9:37', 'Fri-10:42', 'Mon-10:27', 'Sat-10:36', 'Sun-10:33', 'Thu-10:35', 'Tue-10:38', 'Wed-10:31', 'Fri-11:10', 'Mon-11:5', 'Sat-11:18', 'Sun-11:16', 'Thu-11:8', 'Tue-11:4', 'Wed-11:11', 'Fri-12:1', 'Mon-12:3', 'Sat-12:3', 'Sun-12:9', 'Thu-12:7', 'Tue-12:5', 'Wed-12:6', 'Fri-13:2', 'Mon-13:2', 'Sat-13:4', 'Sun-13:8', 'Thu-13:4', 'Tue-13:3', 'Wed-13:3', 'Fri-14:2', 'Mon-14:3', 'Sat-14:8', 'Sun-14:12', 'Thu-14:2', 'Tue-14:5', 'Wed-14:3', 'Fri-15:4', 'Mon-15:4', 'Sat-15:11', 'Sun-15:6', 'Thu-15:8', 'Tue-15:2', 'Wed-15:6', 'Fri-16:4', 'Mon-16:3', 'Sat-16:6', 'Sun-16:10', 'Thu-16:6', 'Tue-16:3', 'Wed-16:3', 'Mon-17:3', 'Sat-17:6', 'Sun-17:15', 'Thu-17:3', 'Tue-17:4', 'Wed-17:5', 'Fri-18:3', 'Mon-18:3

### c) yelp_academic_dataset_review_train.csv

In [14]:
review_train = []
with open('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_review_train.csv',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        review_train.append(tuple(line))

In [15]:
len(review_train)

116475

In [16]:
len(review_train[0]) == len(review_train[1]) 

True

In [17]:
review_train[0]

('',
 'funny',
 'user_id',
 'review_id',
 'text',
 'business_id',
 'stars',
 'date',
 'useful',
 'type',
 'cool')

In [18]:
review_train[1]

('2546',
 '1',
 '_5FF5NN5kHZmGTNuJwpnhg',
 'egN-7YtD9vAfOsSqdsGwlQ',
 "I almost got sick on all of the chocolate coffee beans in this place, they are so good.  I sat outside and had a great conversation about the direction music is taking in this day in age.  If you don't follow a formula it seems that you can't get a record contract, even in the indie scene.  As you can tell, I thoroughly enjoyed the crowd here.  \n\nInside there is a bar opposite the coffee line with Creative Loafing and other local newspapers to read, as well as chocolate coffee beans in little quarter machines.  Don't mix these with a strong cup of coffee or you will be jumping off the walls.",
 'F1tOtPzcsQk8PqNOatVsCg',
 '3',
 '2008-09-11',
 '0',
 'review',
 '0')

### d) yelp_academic_dataset_tip_train.txt

In [20]:
tip_train = []
with open('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_tip.csv',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        tip_train.append(tuple(line))

In [21]:
len(tip_train)

27302

In [22]:
tip_train[0]

('', 'user_id', 'text', 'business_id', 'likes', 'date', 'type')

In [23]:
tip_train[1]

('87',
 'QGgWWhEi5R4SLAKN-xwtNQ',
 'The salsa is delicious... Plan to eat a lot!',
 'xUvgrDdKhcmaP6Zv0ssb6A',
 '0',
 '2014-03-07',
 'tip')

### e) yelp_academic_dataset_user.csv

In [24]:
user = []
with open('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_user.csv',encoding="utf8") as f:
    reader = csv.reader(f, delimiter=',',quoting=csv.QUOTE_NONE)
    for line in reader:
        user.append(tuple(line))

In [30]:
## Data analysis

In [1]:
df = pd.read_csv('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_business_train.csv')

NameError: name 'pd' is not defined

In [ ]:
df = df.dropna(subset=['attributes'])

In [ ]:
def fixdata(df):
    words = {
    "no" : 0, "False":0, "free":1, "full_bar": 1, "casual": 0, "True":1, "none": 0, "beer_and_wine" : 1,"yes_free":1, 
    "average":1,"loud":.75,"quiet":1,"very_loud":0,"yes":1,"outdoor":1,"paid":0.5, "dressy":1,'1':1,"2":2,"3":3,"4":4,
    "yes_corkage":1, "formal":1}
    for i in df.index:
        dicl = dictionize(df['attributes'][[i]][i], "_")
        for j in dicl.keys():
            if j in df.columns:
                if type(dicl[j]) == int:
                    k = dicl[j]
                else:
                    k = words[dicl[j]]
                    
                df.set_value(i,j,k)
            else:
                df[j] = np.zeros(df.shape[0])
    return df


In [ ]:
def fixdata2(df):
    words = {
    "no" : 0, "False":0, "free":1, "full_bar": 1, "casual": 0, "True":1, "none": 0, "beer_and_wine" : 1,"yes_free":1, 
    "average":1,"loud":.75,"quiet":1,"very_loud":0,"yes":1,"outdoor":1,"paid":0.5, "dressy":1,'1':1,"2":2,"3":3,"4":4,
    "yes_corkage":1, "formal":1}
    for i in df.index:
        dicl = dictionize(df['attributes'][[i]][i], "_")
        for j in dicl.keys():
            k = dicl[j]
            if j in df.columns:
                df.set_value(i,j,k)
            else:
                df[j] = list("0" for i in range(0,df.shape[0]))
                df.set_value(i,j,k)
    return df

In [ ]:
def dictionize(column,delim):
    val = {}
    lst = column[1:-1].split(",")
    contains2 = 0
    for j in range(0,len(lst)):
        lst2 = lst[j].split(": ")
        first_contains = 0
        for k in range(0, len(lst2)):
            if '{' in lst2[k]:
                first_contains = 1
                contains2 = 1
                keep = stripignore(lst2[0])
            elif '}' in lst2[k]:
                contains2 = 0
            else:
                k = 9
        if first_contains == 1:
            val[keep + delim + stripignore(lst2[1])] = stripignore(lst2[2])
        elif contains2 == 1:
            val[keep + delim + stripignore(lst2[0])] = stripignore(lst2[1])
        else:
            if '}' in lst2[1]:
                val[keep + delim + stripignore(lst2[0])] = stripignore(lst2[1]) 
            else:
                val[stripignore(lst2[0])] = stripignore(lst2[1])
    return val

                

In [ ]:
def stripignore(string):
    new_string =""
    ignore = ['"',' ', '{',"'", '}']
    for i in range(0,len(string)):
        if string[i] in ignore:
            k = 9
        else:
            new_string = new_string + string[i]
    return(new_string)
        

In [ ]:
new_df = fixdata2(df)
new_df

In [ ]:
new_df.columns

In [ ]:
new_df.shape

In [ ]:
new_df.describe()

In [ ]:
new_df.groupby('Alcohol').aggregate(len)['hours'][0]

In [ ]:
def percent_reporting(frame,start):
    store = {}
    for i in frame.columns[17:len(frame.columns)]:       
        summary = frame.groupby(i).aggregate(len)
        store[i] = 1 - summary['hours'][0]/len(frame)
    return(pd.Series(store))

In [ ]:
dicl = dictionize(df['attributes'][[0]][0], "_")

In [ ]:
def drop_columns(frame, threshold, begin):
    hi = percent_reporting(frame,begin)
    choose = pd.Series({i:1 for i in frame.columns[0:begin]})
    choose = choose.append(hi)
    choose2 = choose > threshold
    for i in frame.columns:
        if i not in choose2[choose2]:
            frame.drop(i, axis=1, inplace=True)
    return(frame)

In [215]:
new_df2 = drop_columns(new_df,.58,17)

We want only the columns that have at least 60% of the data reporting: Therefore, we will drop many of the columns.

new_df2.drop('Unnamed: 0', axis=1, inplace=True)
new_df2.drop('neighborhood', axis=1, inplace=True)
new_df2.drop('hours', axis=1, inplace=True)
new_df2.drop('address', axis=1, inplace=True)
new_df2.drop('city', axis=1, inplace=True)
new_df2.drop('name', axis=1, inplace=True)
new_df2.drop('state', axis=1, inplace=True)
new_df2.drop('longitude', axis=1, inplace=True)
new_df2.drop('latitude', axis=1, inplace=True)

In [231]:
new_df2.columns[17:51]

Index(['Alcohol', 'Ambience_romantic', 'Ambience_intimate', 'Ambience_classy',
       'Ambience_hipster', 'Ambience_divey', 'Ambience_touristy',
       'Ambience_trendy', 'Ambience_upscale', 'Ambience_casual', 'BikeParking',
       'BusinessAcceptsCreditCards', 'BusinessParking_garage',
       'BusinessParking_street', 'BusinessParking_validated',
       'BusinessParking_lot', 'BusinessParking_valet', 'GoodForKids',
       'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch',
       'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch',
       'HasTV', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'WiFi'],
      dtype='object')

In [249]:
def chin_dummies(frame, keyword1, keyword2,drop_list):
    frame = pd.get_dummies(frame, columns = frame.columns[frame.columns.get_loc(keyword1):frame.columns.get_loc(keyword2)])
    frame = drop_columns2(frame, drop_list)
    return(frame)

In [250]:
def drop_columns2(frame,drop_list):
    for i in frame.columns:
        if i in drop_list:
            frame.drop(i, axis=1, inplace=True)
    return(frame)

In [251]:
new_df3 = chin_dummies(new_df2, "Alcohol","WiFi",new_df2.columns[17:51])

In [253]:
"WiFi" in new_df3.columns

False